In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="yAGqrP4bbcFc0kgTqBks")
project = rf.workspace("aarhus-universitet-r1lgs").project("deep-learning-wnuyb")
version = project.version(12)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deep-Learning-12 in yolov11:: 100%|██████████| 1128/1128 [00:00<00:00, 2943.37it/s]


In [7]:
from pathlib import Path

dataset_path = Path("./Deep-learning-12/train") 
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

In [41]:
import yaml

yaml_file = "./Deep-learning-12/data.yaml"  
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes)

# change to index
cls_idx = {cls: idx for idx, cls in enumerate(cls_idx)}
keys = list(cls_idx.values())
print(keys)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [42]:
import pandas as pd

indx = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [43]:
print(labels_df)

                                                   aeble  aeg agurk appelsin  \
AEBLER_I_POSE_png.rf.a73452dcbe3095dda703647be2...   NaN  NaN   NaN      NaN   
AEGTE_MAYONNAISE_png.rf.4c4f2cfa18214e9af5aad1e...   NaN  NaN   NaN      NaN   
AEble_Belle_de_Boskoop_oko-_1_png.rf.2e9d1f7f78...   NaN  NaN   NaN      NaN   
AEble_Belle_de_Boskoop_stor_1_png.rf.5a09d80e53...   NaN  NaN   NaN      NaN   
AEble_Royal_Gala_1_png.rf.3c680fb79e6491fc6b4e4...   NaN  NaN   NaN      NaN   
...                                                  ...  ...   ...      ...   
images_jpg.rf.aab537fa8ad02a17ae4da3028a9577dc       NaN  NaN   NaN      NaN   
images_jpg.rf.af5f769c9f1cbf9ae739b038a314bc08       NaN  NaN   NaN      NaN   
piske_jpg.rf.10c76dabef39c461cb23354afdd2e99d        NaN  NaN   NaN      NaN   
piske_jpg.rf.cc045bca5130c7e19b85793579cd0dcb        NaN  NaN   NaN      NaN   
smagstest-af-rugbroed_jpg.rf.4315ba178c969b3f3f...   NaN  NaN   NaN      NaN   

                                       

In [44]:
from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ")[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

/var/folders/lq/v36xrkmd52s39s9cz2xzy7t80000gn/T/ipykernel_10640/4074969495.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`


In [45]:
# display non-zero values
print(labels_df)


                                                    aeble  aeg  agurk  \
AEBLER_I_POSE_png.rf.a73452dcbe3095dda703647be2...    0.0  0.0    0.0   
AEGTE_MAYONNAISE_png.rf.4c4f2cfa18214e9af5aad1e...    0.0  0.0    0.0   
AEble_Belle_de_Boskoop_oko-_1_png.rf.2e9d1f7f78...    0.0  0.0    0.0   
AEble_Belle_de_Boskoop_stor_1_png.rf.5a09d80e53...    0.0  0.0    0.0   
AEble_Royal_Gala_1_png.rf.3c680fb79e6491fc6b4e4...    0.0  0.0    0.0   
...                                                   ...  ...    ...   
images_jpg.rf.aab537fa8ad02a17ae4da3028a9577dc        0.0  0.0    0.0   
images_jpg.rf.af5f769c9f1cbf9ae739b038a314bc08        0.0  0.0    0.0   
piske_jpg.rf.10c76dabef39c461cb23354afdd2e99d         0.0  0.0    0.0   
piske_jpg.rf.cc045bca5130c7e19b85793579cd0dcb         0.0  0.0    0.0   
smagstest-af-rugbroed_jpg.rf.4315ba178c969b3f3f...    0.0  0.0    0.0   

                                                    appelsin  banan  \
AEBLER_I_POSE_png.rf.a73452dcbe3095dda703647be2...  

In [1]:
from ultralytics import YOLO
import torch

# Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = model.to(device)
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="Deep-Learning-12/data.yaml", epochs=10)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
#results = model("https://ultralytics.com/images/bus.jpg")


Matplotlib is building the font cache; this may take a moment.
/Users/NichlasKondrup/Desktop/Deep Learning/2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/NichlasKondrup/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=Deep-Learning-12/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source

RuntimeError: Dataset 'Deep-Learning-12/data.yaml' error ❌ 
Dataset 'Deep-Learning-12/data.yaml' images not found ⚠️, missing path '/Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-Learning-12/valid/images'
Note dataset download directory is '/Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets'. You can update this in '/Users/NichlasKondrup/Library/Application Support/Ultralytics/settings.json'

In [29]:
results = model.predict(source="data/images",conf=0.2)



image 1/1 c:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\data\images\lettuce_test.jpg: 512x640 1 hakket oksekod, 1 salat, 26.0ms
Speed: 0.0ms preprocess, 26.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)


In [35]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print(boxes)
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 7., 17.], device='cuda:0')
conf: tensor([0.32103, 0.27564], device='cuda:0')
data: tensor([[4.19142e+01, 4.11390e+00, 6.25132e+02, 5.31442e+02, 3.21026e-01, 7.00000e+00],
        [2.90360e+01, 3.79087e+00, 6.29693e+02, 5.30580e+02, 2.75636e-01, 1.70000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (538, 674)
shape: torch.Size([2, 6])
xywh: tensor([[333.52310, 267.77808, 583.21783, 527.32837],
        [329.36441, 267.18546, 600.65680, 526.78912]], device='cuda:0')
xywhn: tensor([[0.49484, 0.49773, 0.86531, 0.98016],
        [0.48867, 0.49663, 0.89118, 0.97916]], device='cuda:0')
xyxy: tensor([[ 41.91417,   4.11390, 625.13202, 531.44226],
        [ 29.03601,   3.79087, 629.69281, 530.58002]], device='cuda:0')
xyxyn: tensor([[0.06219, 0.00765, 0.92750, 0.98781],
        [0.04308, 0.00705, 0.93426, 0.98621]], device='cuda:0')


In [31]:
print(probs)

None
